# Performance evaluation
A. Boulle and A. Souesme, 2025

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from utilities import Log10Layer
%matplotlib ipympl

### Load and format data

In [ ]:
# ======== Begin user input ========
# Path to input data
data_path = "/PATH/TO/DATA/FOLDER"
# Index corresponding to "size" in test&_label
indices_labels = [20]
# Angular range used when creating the dataset. For display purpose only
tth = np.arange(15, 120.01, 0.01)
# ======== End user input ========

# Load test data
sampling = 1
raw_test = np.load(data_path + "/test.npz")
test_data = raw_test["test_data"][:, :]
test_label = raw_test["test_label"][:, :]
test_data = test_data[::sampling, :]
test_label = test_label[::sampling, indices_labels]

# Load trained model
cnn = keras.models.load_model(
    "./checkpoints/CNN.hdf5",
    custom_objects={"Log10Layer": Log10Layer},
)

# Load normalization scheme
with open("./checkpoints/normL.class", "rb") as f:
    normL = pickle.load(f)

### Plot data

In [ ]:
nb_data = np.shape(test_label)[0]
random_data = np.random.randint(nb_data)
print(f"size = {test_label[random_data][0]:.3f} µm")
plt.cla()
plt.semilogy(tth, test_data[random_data, :])
plt.show()

### Inference on test data

In [ ]:
# Preprocess test data and labels
test_data = test_data[:, :, np.newaxis, np.newaxis]
test_label = normL.forward(test_label)

In [ ]:
# Evaluate overall performance
cnn.evaluate(test_data, test_label, batch_size=32)

### Plot predictions _vs_ test data

In [ ]:
# Make predictions on test data
raw_pred = cnn.predict(test_data)
predictions = normL.backwards(raw_pred)
# Denormalize test labels
test_label = normL.backwards(test_label)

# Plot predictions vs true labels
fig, ax = plt.subplots(1, 1, figsize=(3.4, 3.4))
ax.plot(test_label * 1000, predictions * 1000, "o", alpha=0.1)
ax.set_xlabel("True size (µm)")
ax.set_ylabel("Predicted size (µm)")
plt.tight_layout()
plt.show()


# re-normalize labels for multiple cell runs
predictions = normL.forward(raw_pred)
test_label = normL.forward(test_label)